In [1]:
import os
import pandas as pd
import sys
#sys.path.insert(0, os.path.abspath('ComParE2022_VecNet/src'))
sys.path.append(dir)
#import config ,config_pytorch
#from evaluate import get_results
import numpy as np

# Troubleshooting and visualisation
# import IPython.display as ipd



In [2]:
# humbug lib imports
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#from PyTorch import config_pytorch
import math
import pickle



In [3]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
import time
#import datetime



In [4]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import average_precision_score
import sys



In [7]:
from tqdm.notebook import tqdm
# additional pytorch tools
import random
import torchaudio
import torchaudio.transforms as AT
import torchvision.transforms as VT
from torch.cuda.amp import autocast, GradScaler
#from timm.scheduler.cosine_lr import CosineLRScheduler
# import timm
# import timm.optim
# from timm.loss import BinaryCrossEntropy
# from timm.utils import NativeScaler
# from timm.models import model_parameters
from glob import glob
## nnAudio


In [8]:
from nnAudio import features
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import argparse
## DDp Import
import torch.distributed as dist
import deepspeed
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
p = os.path.join("..","..","models","model_e103_2023_05_04_19_08_14.pth")
print(p)

../../models/model_e103_2023_05_04_19_08_14.pth


In [9]:
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

NameError: name '__file__' is not defined

In [8]:
class MyModel(nn.Module):
    def __init__(self, model_name, image_size):
        super().__init__()
        # num_classes=0 removes the pretrained head
        self.backbone = timm.create_model(model_name,
                        pretrained=True, num_classes=8, in_chans=1, 
                        drop_path_rate=0.2, global_pool='max',
                        drop_rate=0.25)
        #####  This section is model specific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
                #### end layer freezing
        self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size),antialias = True)
        self.spec_layer = features.STFT(n_fft=int(2048), freq_bins=None, hop_length=int(128),
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                           sr=8000, output_format="Magnitude", trainable=False,verbose = False)
        self.batch_norm = nn.BatchNorm2d(num_features= 1)
        #self.augment_layer = augment_audio(trainable = True, sample_rate = config.rate)
        
    def forward(self, x,train = True):
        # first compute spectrogram
        warnings.filterwarnings("ignore", category=DeprecationWarning, module="nnAudio")
        spec_gram = self.spec_layer(x)
        #print("post spec gram shape = ",spec_gram.shape)
        spec_gram = self.batch_norm(spec_gram.unsqueeze(dim = 1))
        #print("post norm shape = ",spec_gram.shape)
        spec_gram_nan_check = torch.isnan(spec_gram).any().item()
        assert not (spec_gram_nan_check) ,"Tensor contains NaN values after spec gram creation."
        
        with torch.no_grad():
            if train == True:
                #generate a random number and if condition is met apply aug
                ta_transformations_rndm_choice = VT.RandomChoice([AT.FrequencyMasking(freq_mask_param=100),AT.TimeMasking(time_mask_param=50)], p=[.4, .4])
                ta_transformations_rndm_apply = VT.RandomApply([AT.FrequencyMasking(freq_mask_param=50),AT.TimeMasking(time_mask_param=25)],p = .15)
                spec_gram = ta_transformations_rndm_choice(spec_gram)
                spec_gram = ta_transformations_rndm_apply(spec_gram)
                spec_gram_nan_check = torch.isnan(spec_gram).any().item()
                assert not (spec_gram_nan_check) ,"Tensor contains NaN values after augmentations  "
                
                
            
        
        x = self.sizer(spec_gram.squeeze(dim = 1))
        #print("post sizer shape = ",x.shape)
        x = x.unsqueeze(dim = 1)
        #print("post unsqueeze shape = ",x.shape)
        
        # then repeat channels
        del spec_gram,spec_gram_nan_check
                  
        x = self.backbone(x)
        backbone_op_nan_check = torch.isnan(x).any().item()
        assert not (backbone_op_nan_check) ,"Tensor contains NaN values in the backbone OP "
        #print("x shape = " + str(x.shape))
        #print("x = " +str(x))
        #pred = nn.Softmax(x)
        pred = x
        #print(np.argmax(pred.detach().cpu().numpy()))
        #print(pred)
        output = {"prediction": pred }
        #print(output)
        del x , backbone_op_nan_check
        return output

In [13]:
model_new = MyModel('convnext_xlarge_in22k',224 )

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_xlarge_22k_224.pth


In [14]:
classes = ['an arabiensis','culex pipiens complex', 'ae aegypti','an funestus ss','an squamosus',
               'an coustani','ma uniformis','ma africanus' ]

In [15]:
#weights_adj = torch.tensor(class_weights).type(torch.float).to(device)
criterion_1 = nn.CrossEntropyLoss(label_smoothing=.1)
criterion_2 = nn.CrossEntropyLoss()

In [16]:
lr = .0015

In [17]:
optimiser = timm.optim.AdamP(model_new.parameters(), lr= lr)

In [18]:
import deepspeed

In [25]:
os.environ['MASTER_ADDR'] = 'localhost' 
os.environ['MASTER_PORT'] = '9956' 

In [26]:
dist.init_process_group( backend='nccl', world_size=4, rank=4 ) 

[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to 

KeyboardInterrupt: 

s).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect

In [ ]:
model, _, _, _ = deepspeed.initialize( model=model_new, optimizer=optimiser,
    #optimizer_args={"lr": lr},
    #loss_scale=0.1,
    
)

 socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [localhost]:9956 (errno: 99 - Cannot assign requested address).
[W socket.cpp:601] [c10d] The client socket has failed to connect to [l